In [1]:
import os
import pandas as pd
import pickle

In [2]:
os.listdir('Data/')

['list_attr_celeba.txt',
 '.DS_Store',
 'identity_CelebA.txt',
 'img_align_celeba']

In [8]:
df = pd.read_csv('Data/identity_CelebA.txt', delimiter=' ', header=None)
df.columns = ['image_id', 'client_id']

In [9]:
df.head()

,image_id,client_id
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692
3,000004.jpg,5805
4,000005.jpg,9295


In [43]:
idx = df['client_id'].value_counts()[:10].index

In [19]:
prop_df = pd.read_csv('scalableRR_property_attack/Data/list_attr_celeba.txt', sep=r"\s+", skiprows=1)

In [23]:
prop_df.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [24]:
df.head()

,image_id,client_id
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692
3,000004.jpg,5805
4,000005.jpg,9295


In [25]:
prop_df['image_id'] = prop_df.index
prop_df.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_id
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,1,1,-1,1,-1,1,-1,-1,1,000001.jpg
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,000002.jpg
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,1,000003.jpg
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,-1,1,1,-1,1,000004.jpg
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,000005.jpg


In [27]:
prop_df = prop_df.reset_index(drop=True)

In [28]:
print(df.shape, prop_df.shape)

(202599, 2) (202599, 41)


In [30]:
prop_df['image_id'].dtypes

dtype('O')

In [31]:
df['image_id'].dtypes

dtype('O')

In [33]:
prop_df = prop_df.merge(df, on='image_id')
prop_df.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,image_id,client_id
0,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,1,-1,1,-1,-1,1,000001.jpg,2880
1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,000002.jpg,2937
2,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,1,000003.jpg,8692
3,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,1,-1,1,1,-1,1,000004.jpg,5805
4,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,000005.jpg,9295


In [46]:
prop_df[prop_df['client_id'].isin(idx)].groupby('client_id')['Male'].nunique()

client_id
2820    1
3227    1
3699    1
3745    1
3782    1
4887    1
6568    1
8968    1
9152    2
9256    1
Name: Male, dtype: int64

In [47]:
prop_df[prop_df['client_id'].isin(idx)].groupby('client_id')['Eyeglasses'].nunique()

client_id
2820    1
3227    2
3699    2
3745    1
3782    1
4887    2
6568    1
8968    1
9152    2
9256    1
Name: Eyeglasses, dtype: int64

In [41]:
prop_df.columns

Index(['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young', 'image_id',
       'client_id'],
      dtype='object')

In [49]:
prop_df.corr()['Male']

5_o_Clock_Shadow       0.417670
Arched_Eyebrows       -0.408016
Attractive            -0.394451
Bags_Under_Eyes        0.301077
Bald                   0.178101
Bangs                 -0.163264
Big_Lips              -0.167472
Big_Nose               0.369275
Black_Hair             0.116279
Blond_Hair            -0.302988
Blurry                 0.024389
Brown_Hair            -0.112482
Bushy_Eyebrows         0.247066
Chubby                 0.230486
Double_Chin            0.207514
Eyeglasses             0.202057
Goatee                 0.305602
Gray_Hair              0.184424
Heavy_Makeup          -0.666724
High_Cheekbones       -0.250162
Male                   1.000000
Mouth_Slightly_Open   -0.099604
Mustache               0.246147
Narrow_Eyes            0.013786
No_Beard              -0.522243
Oval_Face             -0.120342
Pale_Skin             -0.077538
Pointy_Nose           -0.217200
Receding_Hairline      0.117389
Rosy_Cheeks           -0.213934
Sideburns              0.289046
Smiling 

In [50]:
prop_df.to_csv('Data/prop.csv', index=False)